Load API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Load Contents from HLB Fixed Deposite Pages

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

Split the Loaded Data into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(data)
texts

Initialize Pinecone

In [ ]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)

index_name = "main-index"
namespace = "hlb-fixed-deposit-with-cohere"

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
  
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

Create Embeddings and Vectors to Insert to Pinecone Database

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import CohereEmbeddings, OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [ ]:
retriever = vector.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9, "k": 15})
retriever

Creating Prompt Template

In [29]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-pro", temperature=0.1)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  Don't need to refer to chat history as long as you have enough information from the context.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['question', 'context'])

Creating Memory

In [30]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", output_key="answer", max_token_limit=500, return_messages=True)

Creating Chain

In [31]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  memory=memory,
  verbose=True,
  combine_docs_chain_kwargs={"prompt": prompt},
  return_source_documents=True
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')
    for source_documents in result["source_documents"]:
      print(f'\nDocument: {source_documents}')


Running the Chain

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")

In [ ]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")

In [ ]:
ask_question("What are the interest rates for normal Fixed Deposit?")

In [35]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Assistant: I apologize, but I cannot answer your question based on the context you provided. The document does not contain information about the total amount that can be withdrawn at the end of the term for a fixed deposit of RM 50,000 for 12 months.
Follow Up Input: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related

In [36]:
ask_question("who is jay z?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Assistant: I apologize, but I cannot answer your question based on the context you provided. The document does not contain information about the total amount that can be withdrawn at the end of the term for a fixed deposit of RM 50,000 for 12 months.
Human: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Assistant: I apologize, but the information you are seeking cannot be found in the context you provided. Therefore, I am unable to answer your question.
Follow Up Input: who is jay z?
Standalone question:

> Finished chai

d:\Code\langchain-test\venv\Lib\site-packages\langchain_core\vectorstores.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: Siapa Jay Z?
  By only basing on this context: 
  Don't need to refer to chat history as long as you have enough information from the context.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as w